In [23]:
from azureml.core import Workspace
ws = Workspace.from_config()
print('Workspace name:'+ ws.name,
       'Azure region :'+ ws.location,
       'Subscription ID:' + ws._subscription_id,
       'Resource group'+ ws.resource_group

)

Workspace name:labuser67ml Azure region :koreacentral Subscription ID:27db5ec6-d206-4028-b5e1-6004dca5eeef Resource grouprg67


In [24]:
# 실험공간을 준비 
from azureml.core import Experiment
experiment = Experiment(workspace=ws, name='diabetes-experiment')

In [25]:
# 데이터를 준비한다.
from azureml.opendatasets import Diabetes
from sklearn.model_selection import train_test_split

x_df = Diabetes.get_tabular_dataset().to_pandas_dataframe().dropna()
x_df # age~s6 을이용해서 y값 맞추기 y가 라벨 age~ x독립변수 x를 학습시켜야 하니 y는 따로 떼야함
y_df = x_df.pop('Y')

X_train ,X_test ,y_train , y_test = train_test_split(x_df,y_df , test_size=0.2,random_state=66)

print(X_train)

     AGE  SEX   BMI     BP   S1     S2    S3    S4      S5   S6
440   36    1  30.0   95.0  201  125.2  42.0  4.79  5.1299   85
389   47    2  26.5   70.0  181  104.8  63.0  3.00  4.1897   70
5     23    1  22.6   89.0  139   64.8  61.0  2.00  4.1897   68
289   28    2  31.5   83.0  228  149.4  38.0  6.00  5.3132   83
101   53    2  22.2  113.0  197  115.2  67.0  3.00  4.3041  100
..   ...  ...   ...    ...  ...    ...   ...   ...     ...  ...
122   62    2  33.9  101.0  221  156.4  35.0  6.00  4.9972  103
51    65    2  27.9  103.0  159   96.8  42.0  4.00  4.6151   86
119   53    1  22.0   94.0  175   88.0  59.0  3.00  4.9416   98
316   53    2  27.7   95.0  190  101.8  41.0  5.00  5.4638  101
20    35    1  21.1   82.0  156   87.8  50.0  3.00  4.5109   95

[353 rows x 10 columns]


In [26]:
# 모델 훈련 , 로그, 파일관리 
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
from sklearn.externals import joblib # 우리가 학습시킨걸 저장이 가능하다
import math

alphas = [0.1, 0.2 ,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1]

for alpha in alphas:
    # 실험의 기록
    run = experiment.start_logging()
    run.log('alpha_value', alpha)

    model = Ridge(alpha=alpha)
    model.fit(X_train , y_train)
    y_pred = model.predict(X_test)
    rmse = math.sqrt(mean_squared_error(y_test, y_pred))

    run.log('rmse',rmse)

    print('model_alpha={0}, rmse = {1}'.format(alpha,rmse))

    model_name = 'model_alpha_'+ str(alpha) + '.pkl' # 
    filename = 'outputs/'+ model_name

    joblib.dump(value=model, filename=filename)
    #Azure ML Service에 모델파일 업로드 하는 부분
    run.upload_file(name=model_name, path_or_stream=filename)

    run.complete()

    print(f'{alpha} experiment complted')

model_alpha=0.1, rmse = 56.60520331339142
0.1 experiment complted
model_alpha=0.2, rmse = 56.61060264545032
0.2 experiment complted
model_alpha=0.3, rmse = 56.616243245483616
0.3 experiment complted
model_alpha=0.4, rmse = 56.622107088710145
0.4 experiment complted
model_alpha=0.5, rmse = 56.62817734275138
0.5 experiment complted
model_alpha=0.6, rmse = 56.63443828302745
0.6 experiment complted
model_alpha=0.7, rmse = 56.64087521475942
0.7 experiment complted
model_alpha=0.8, rmse = 56.64747440101076
0.8 experiment complted
model_alpha=0.9, rmse = 56.654222996253125
0.9 experiment complted
model_alpha=1, rmse = 56.66110898499056
1 experiment complted


In [27]:
experiment

Name,Workspace,Report Page,Docs Page
diabetes-experiment,labuser67ml,Link to Azure Machine Learning studio,Link to Documentation


In [19]:
# Best Model을 탐색 후 다운로드
minimum_rmse = None
minimum_rmse_runid = None

for exp in experiment.get_runs():
    run_metrics = exp.get_metrics();
    run_details = exp.get_details();

    run_rmse = run_metrics['rmse']
    run_id = run_details['runId']

    # 가장 낮은 RMES 값을 가진 실행 ID를 구하는 부분
    if minimum_rmse is None: # 제일처음 실행시 None 위에서 설정
        minimum_rmse = run_rmse
        minimum_rmse_runid = run_id
    else:
        if run_rmse < minimum_rmse:
            minimum_rmse = run_rmse
            minimum_rmse_runid = run_id

    
print('Best run_id:'+minimum_rmse_runid)
print('Best run_id_rmse:'+str(minimum_rmse))




Best run_id:7186405d-efda-4dda-82a5-30d2d1ac0f71
Best run_id_rmse:56.60520331339142


In [28]:
from azureml.core  import Run
best_run = Run(experiment=experiment , run_id=minimum_rmse_runid)
print(best_run.get_file_names())

best_run.download_file(name=str(best_run.get_file_names()[0]))

ServiceException: ServiceException:
	Code: 404
	Message: (UserError) Run 7186405d-efda-4dda-82a5-30d2d1ac0f71 was not found
	Details:

	Headers: {
	    "Date": "Thu, 04 May 2023 08:35:33 GMT",
	    "Content-Type": "application/json; charset=utf-8",
	    "Transfer-Encoding": "chunked",
	    "Connection": "keep-alive",
	    "Vary": "Accept-Encoding",
	    "Request-Context": "appId=cid-v1:4f6105a5-f89a-4844-a2b0-5b2e6cbb3d2c",
	    "x-ms-response-type": "error",
	    "Strict-Transport-Security": "max-age=15724800; includeSubDomains; preload",
	    "X-Content-Type-Options": "nosniff",
	    "x-aml-cluster": "vienna-koreacentral-02",
	    "x-request-time": "0.103",
	    "Content-Encoding": "gzip"
	}
	InnerException: {
    "additional_properties": {},
    "error": {
        "additional_properties": {
            "debugInfo": null
        },
        "code": "UserError",
        "severity": null,
        "message": "Run 7186405d-efda-4dda-82a5-30d2d1ac0f71 was not found",
        "message_forma